In [24]:
from datetime import datetime, timedelta
from binance.client import Client
import pandas as pd
import pickle



def get_client():
    fn = 'C:\\Users\\FUJSC\\Documents\\key\\binance-key.pickle'
    with open(fn, 'rb') as handle:
        k = pickle.load(handle)
    return Client(k['API_KEY'], k['API_SECRET'])


def get_btc_futures_price(start_time, end_time):
    # Create a Binance Futures client instance
    client = get_client()

    # Convert the start_time and end_time strings to datetime objects
    start_datetime = datetime.strptime(start_time, '%d-%m-%Y %H:%M:%S')
    end_datetime = datetime.strptime(end_time, '%d-%m-%Y %H:%M:%S')

    # Initialize a list to store the klines
    klines = []

    # Define the interval as 15 minutes
    interval = "15m"

    # Fetch the klines (candlestick data) from Binance Futures API
    while start_datetime < end_datetime:
        # Calculate the end time as 15 minutes after the start time
        end_time = start_datetime + timedelta(minutes=15)

        # Convert the datetime objects to UNIX timestamps
        start_timestamp = int(start_datetime.timestamp() * 1000)
        end_timestamp = int(end_time.timestamp() * 1000)

        # Fetch the klines for the current interval
        current_klines = client.futures_klines(symbol="BTCUSDT", interval=interval,
                                               startTime=start_timestamp, endTime=end_timestamp)
        klines.extend(current_klines)

        # Update the start time for the next iteration
        start_datetime = end_time

    # Extract the high and low prices from the klines
    high_price = float(max(klines, key=lambda x: float(x[2]))[2])  # High price is at index 2
    low_price = float(min(klines, key=lambda x: float(x[3]))[3])   # Low price is at index 3

    return high_price, low_price

def format_data():
    fn = 'C:\\Users\\FUJSC\\Downloads\\GRIND_BTC_15.csv'
    raw = pd.read_csv(fn)
    entry_time = list(raw['DateTime'])
    signal = list(raw['Signal'])
    price = list(raw['Price'])
    exit_time = [entry_time[idx+1] for idx in range(len(entry_time)-1)]+[None]
    raw['entry_time'] = entry_time
    raw['exit_time'] = exit_time
    pnl=[]
    for idx in range(5):
        high_price, low_price = get_btc_futures_price(entry_time[idx], exit_time[idx])
        if signal[idx] == 'S':
            pnl.append(round((price[idx]*100/low_price)-100, 2))
        if signal[idx] == 'L':
            pnl.append(round((high_price*100/price[idx])-100, 2))
    print(pnl)
    raw.to_csv('formatted.csv', index=False)
    return raw





In [ ]:
start_time = "13-06-2023 16:00:00"
end_time = "13-06-2023 17:00:00"
high_price, low_price = get_btc_futures_price(start_time, end_time)
print(f"High Price: {high_price}")
print(f"Low Price: {low_price}")


In [25]:
r = format_data()
print(r)

[1.11, -0.06, 0.72, 0.29, 2.2]
    Signal             DateTime    Price           entry_time  \
0        S  03-03-2023 06:15:00  22366.0  03-03-2023 06:15:00   
1        L  04-03-2023 17:00:01  22376.0  04-03-2023 17:00:01   
2        S  04-03-2023 22:15:01  22307.3  04-03-2023 22:15:01   
3        L  05-03-2023 05:30:03  22546.0  05-03-2023 05:30:03   
4        S  07-03-2023 12:15:02  22390.1  07-03-2023 12:15:02   
..     ...                  ...      ...                  ...   
101      L  07-06-2023 15:45:02  26816.4  07-06-2023 15:45:02   
102      S  07-06-2023 18:45:05  26490.2  07-06-2023 18:45:05   
103      L  08-06-2023 16:45:02  26478.0  08-06-2023 16:45:02   
104      S  09-06-2023 23:15:02  26424.2  09-06-2023 23:15:02   
105      L  11-06-2023 20:45:01  25931.7  11-06-2023 20:45:01   

               exit_time  
0    04-03-2023 17:00:01  
1    04-03-2023 22:15:01  
2    05-03-2023 05:30:03  
3    07-03-2023 12:15:02  
4    07-03-2023 22:45:01  
..                   ...  